# Урок 11. Модель Transformer-1
## Задание
1. Взять предобученную трансформерную архитектуру и решить задачу перевода (для тогоже корпуса что вы выбрали из предыдущего дз)
2. Cкачиваем из
https://github.com/natasha/corus/blob/master/README.md
любой новостной датасет

    Реализовать метод поиска ближайших статей
    (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру топ 5-ть или 3-ри, ваш метод должен возвращать топ-k ближайших статей к этому запросу).
    Визуально оценить качество

In [1]:
!pip install transformers
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
! pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Корпус предложений из предыдущего дз:

In [4]:
corp = ['Привет!', 'Как дела?', 'Как тебя зовут?', 'Что ты делаешь завтра?', 'Я собираюсь играть в футбол',
        'Если я смогу, я приду играть', 'Когда мы играли в футбол, шел дождь', 'В последнее время я поздно ложусь спать',
        'Проезжая мимо станции, у меня слетела шляпа', 'Ёж птица гордая, пока не пнешь - не полетит']

Возьмем первую модель для перевода с русского на английский на https://huggingface.co/: Helsinki-NLP/opus-mt-ru-en

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

In [6]:
def remove_char_trans(result):
    remove_result_trans = result[6 : -1]
    return remove_result_trans

In [7]:
import pandas as pd

In [8]:
final_result = []

def translate(sentence):

  inputs = tokenizer.encode(sentence, return_tensors="pt")
  outputs = model.generate(inputs, max_length=40, num_beams=4, early_stopping=True)

  result = tokenizer.decode(outputs[0])
  res = (remove_char_trans(result))
  
  final_result.append([sentence, res])
  df = pd.DataFrame(final_result, columns=['текст', ' трансформер'])

  return df

In [9]:
translate('Привет!')
translate('Как дела?')
translate('Как тебя зовут?')
translate('Что ты делаешь завтра?')
translate('Я собираюсь играть в футбол')
translate('Если я смогу, я приду играть')
translate('Когда мы играли в футбол, шел дождь')
translate('В последнее время я поздно ложусь спать')
translate('Проезжая мимо станции, у меня слетела шляпа')
translate('Еж  птица гордая, пока не пнешь - не полетит')

,текст,трансформер
0,Привет!,Hey
1,Как дела?,How's it going
2,Как тебя зовут?,What's your name
3,Что ты делаешь завтра?,What are you doing tomorrow
4,Я собираюсь играть в футбол,I'm gonna play football
5,"Если я смогу, я приду играть","If I can, I'll come play"
6,"Когда мы играли в футбол, шел дождь","When we played football, it rained"
7,В последнее время я поздно ложусь спать,I've been going to bed lately
8,"Проезжая мимо станции, у меня слетела шляпа","As I passed the station, my hat fell off"
9,"Еж птица гордая, пока не пнешь - не полетит","The bird is proud until you kick it, it'll fly"


По переводу видно, что переводчик не понял только слово ёж. Все остальное переводит очень хорошо.

## Задание 2

In [10]:
!wget https://www.dropbox.com/s/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz

--2022-09-19 23:22:34--  https://www.dropbox.com/s/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz [following]
--2022-09-19 23:22:35--  https://www.dropbox.com/s/raw/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfdb8c0979bc70fd8b79949658a.dl.dropboxusercontent.com/cd/0/inline/BtTnD2VYZIM5MgEdhVBwpCuwagt90hBkE3FlMM00ggBub-KNPaM6aT_8_IQ-tUNj2fBejvqX1eyPIwQHhnzu4QDclDi2EBNWBPGzRT-NR20i7uknHy5X10Q_EyCasCKC9kJmvJkSkY5yg8OLD3uMuDdc5BZuUdw6mDNgB1AL6iNfqw/file# [following]
--2022-09-19 23:22:35--  https://ucfdb8c0979bc70fd8b79949658a.dl.dropboxusercontent.com/cd/0/inline/BtTnD2VYZIM5MgEdhVBwpCuwagt90hBkE3FlMM00ggBub-KNPaM6aT_8

In [11]:
!sudo tar -xvf gazeta_jsonl.tar.gz

gazeta_train.jsonl
gazeta_test.jsonl
gazeta_val.jsonl


In [12]:
!head -5 gazeta_train.jsonl

{"url": "https://www.gazeta.ru/financial/2011/11/30/3852658.shtml", "text": "«По итогам 2011 года чистый отток может составить примерно $80 млрд, в следующем году — около $20 млрд. При этом мы ожидаем, что со второго полугодия 2012 года начнется приток капитала», — заявил «Интерфаксу» замминистра экономического развития Андрей Клепач. Официальные прогнозы по выводу капитала из России становятся все пессимистичными: еще летом власти полагали, что из страны уйдет не более $35 млрд, в сентябре Минэкономразвития назвал цифру $50 млрд, в начале ноября Центробанк пересмотрел оценку до $70 млрд. Очередное изменение прогноза было ожидаемо: по расчетам Центробанка , за январь — октябрь чистый отток капитала достиг $64 млрд, причем в последние месяцы он ускорился: в сентябре он составил $14 млрд, в октябре — $13 млрд против среднего ежемесячного оттока в $6—8 млрд в первом полугодии. «После октябрьских данных Минэкономразвития вынуждено было изменить оценку, настаивать на $70 млрд означало ожида

In [13]:
import json

data = []
with open('gazeta_train.jsonl', 'r') as json_file:
    json_list = list(json_file)
for json_str in json_list:
    data.append(json.loads(json_str))

In [14]:
data[0]

{'url': 'https://www.gazeta.ru/financial/2011/11/30/3852658.shtml',
 'text': '«По итогам 2011 года чистый отток может составить примерно $80 млрд, в следующем году — около $20 млрд. При этом мы ожидаем, что со второго полугодия 2012 года начнется приток капитала», — заявил «Интерфаксу» замминистра экономического развития Андрей Клепач. Официальные прогнозы по выводу капитала из России становятся все пессимистичными: еще летом власти полагали, что из страны уйдет не более $35 млрд, в сентябре Минэкономразвития назвал цифру $50 млрд, в начале ноября Центробанк пересмотрел оценку до $70 млрд. Очередное изменение прогноза было ожидаемо: по расчетам Центробанка , за январь — октябрь чистый отток капитала достиг $64 млрд, причем в последние месяцы он ускорился: в сентябре он составил $14 млрд, в октябре — $13 млрд против среднего ежемесячного оттока в $6—8 млрд в первом полугодии. «После октябрьских данных Минэкономразвития вынуждено было изменить оценку, настаивать на $70 млрд означало ожид

In [15]:
len(data)

52400

In [16]:
! pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import annoy
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

In [18]:
columns = ['date', 'title', 'summary', 'text', 'url']
data_df = {col: [] for col in columns}
for itm in data:
  for col in columns:
    data_df[col].append(itm[col])
data_df = pd.DataFrame(data_df, columns=columns)

data_df

,date,title,summary,text,url
0,2011-11-30 18:33:39,Прогноз не успевает за оттоком,"В 2011 году из России уйдет $80 млрд, считают ...",«По итогам 2011 года чистый отток может состав...,https://www.gazeta.ru/financial/2011/11/30/385...
1,2013-01-24 18:20:09,Google закончил поиск,"Юлия Соловьева, экс-директор холдинга «Профмед...",Российское подразделение интернет-корпорации G...,https://www.gazeta.ru/business/2013/01/24/4939...
2,2018-02-06 21:21:14,«Фигуранты дела могут давить на свидетелей»,Суд арестовал на два месяца четверых экс-чинов...,Басманный районный суд Москвы вечером 6 феврал...,https://www.gazeta.ru/social/2018/02/06/116393...
3,2013-06-21 17:43:50,«С последних традиционно «отжимают» больше»,Мнения предпринимателей по поводу вступления в...,Как повлияло вступление в ВТО на конкурентносп...,https://www.gazeta.ru/business/2013/06/21/5388...
4,2014-12-27 01:10:01,Третий «Голос» за Градского,На Первом канале завершился третий сезон шоу «...,К третьему сезону «Голос» на Первом канале ста...,https://www.gazeta.ru/culture/2014/12/27/a_636...
...,...,...,...,...,...
52395,2010-10-13 14:01:03,Взятку привезли на «Мерседесе»,В Москве за вымогательство $700 тысяч задержан...,Следователя следственного отдела московского р...,https://www.gazeta.ru/social/2010/10/13/342826...
52396,2011-01-24 23:30:01,Аэропорты вызывают на досмотр,Сразу после теракта в московском аэропорту Дом...,В понедельник из-за взрыва в Домодедово в оста...,https://www.gazeta.ru/social/2011/01/24/350307...
52397,2014-05-19 15:02:31,Тимберлейк не увидел Майкла Джексона,Вручены награды музыкального журнала Billboard...,В Лас-Вегасе прошла церемония награждения побе...,https://www.gazeta.ru/culture/2014/05/19/a_603...
52398,2013-08-21 15:14:43,Крымские чиновники уедут на поселение,Абинский райсуд Краснодарского края вынес приг...,В среду Абинский райсуд Краснодарского края пр...,https://www.gazeta.ru/social/2013/08/21/560047...


In [19]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer

Возьмем из методички бертовую модель для построения эмбеддинга текста

In [20]:
bert = TFAutoModel.from_pretrained("Geotrend/bert-base-ru-cased")
tokenizer = AutoTokenizer.from_pretrained("Geotrend/bert-base-ru-cased")

Some layers from the model checkpoint at Geotrend/bert-base-ru-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at Geotrend/bert-base-ru-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [59]:
def preprocess_batch(batch):
    token = tokenizer(batch, 
                      max_length=50, 
                      truncation=True, 
                      padding='max_length', 
                      return_token_type_ids=False, 
                      return_tensors='tf')
    return token

In [60]:
def get_tokens(corpus):
    num_batches = len(corpus) // 512
    last_batch = False
    if len(corpus) % 512 > 0:
        last_batch = True
    tokens = []
    for idx in tqdm_notebook(range(num_batches)):
        token = preprocess_batch(corpus[idx * 512:(idx + 1) * 512])
        tokens.append(token)
    if last_batch:
        tokens.append(preprocess_batch(corpus[num_batches * 512:]))
    return tokens

In [61]:
sent_summary = get_tokens(data_df.summary.tolist())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/102 [00:00<?, ?it/s]

In [62]:
def get_embeddings(model, tokens):
    embeddings = []
    for token in tqdm_notebook(tokens):
        embedding = model(**token)[1]
        embeddings.extend(embedding)
    
    return embeddings

In [63]:
embeddings = get_embeddings(bert, sent_summary)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/103 [00:00<?, ?it/s]

Посмотрим на размерность вектора эмбеддинга предложения

In [64]:
embeddings[0].shape

TensorShape([768])

In [65]:
len(embeddings)

52400

In [66]:
# Через AnnoyIndex не получилось - векторы размерности 768 обрабатывались 5 часов
"""
def get_annoy_index(embeddings):
    index = annoy.AnnoyIndex(768 ,'angular')
    
    counter = 0

    for vector in tqdm_notebook(embeddings):
        index.add_item(counter, vector)
        counter += 1
        
    index.build(10)

    return index
    """

"\ndef get_annoy_index(embeddings):\n    index = annoy.AnnoyIndex(768 ,'angular')\n    \n    counter = 0\n\n    for vector in tqdm_notebook(embeddings):\n        index.add_item(counter, vector)\n        counter += 1\n        \n    index.build(10)\n\n    return index\n    "

Так как через Annoy не получилось, просто посчитаем евклидово расстояние между вектором запроса и эмбеддингами аннотаций статей

In [67]:
from scipy.spatial import distance

def get_response(question, embeddings, model, most_similar=3):
    question = preprocess_batch(question)
    vector = model(**question)[1]

    dists = distance.cdist(vector, embeddings, metric='euclidean')[0]
    sd = np.sort(dists)[:most_similar]
    answers = []
    for itm in sd:
        answers.append(np.where(dists == itm)[0][0])
    return answers

In [68]:
question = 'Чемпион России по футболу'
answers = get_response(question, embeddings, bert)
data_df.iloc[answers]

,date,title,summary,text,url
27502,2012-01-26 00:08:44,Пантелич нацелился на Кубок,"«Олимпиакос», разгромив дома «Паниониос», выше...",Соперник казанского «Рубина» по 1/16 финала Ли...,https://www.gazeta.ru/sport/2012/01/25/a_39749...
30436,2013-04-20 01:03:26,«Ткачи» не ровня «Ворскле»,Ведущий борьбу за путевку в Лигу чемпионов «Ли...,"Не только в России, но и во многих европейских...",https://www.gazeta.ru/sport/2013/04/20/a_52627...
5973,2012-09-19 16:35:00,Овечкин пойдет в атаку с бело-голубыми,Нападающий Александр Овечкин на время локаута ...,Капитан «Вашингтона» Александр Овечкин вернулс...,https://www.gazeta.ru/sport/2012/09/19/a_47784...


In [69]:
question = 'Программы освоения космоса'
answers = get_response(question, embeddings, bert)
data_df.iloc[answers]

,date,title,summary,text,url
31490,2010-06-10 15:57:34,Рейтинг падает от усталости,Рейтинги доверия президенту Дмитрию Медведеву ...,«Фонд общественное мнение» опубликовал результ...,https://www.gazeta.ru/politics/2010/06/10_a_33...
14649,2019-04-16 20:05:33,"«Сроки переносятся, расходы растут»: Путин про...","Сроки проектов переносятся, бюджетные расходы ...",О новых задачах и проблемах в российской косми...,https://www.gazeta.ru/science/2019/04/16_a_123...
17521,2010-11-30 12:00:37,Пхеньян развел центрифуги,"Северная Корея заявляет, что имеет «тысячи раб...",На заводе по обогащению урана в КНДР запущены ...,https://www.gazeta.ru/politics/2010/11/30_a_34...


In [72]:
question = 'новости IT сферы'
answers = get_response(question, embeddings, bert)
data_df.iloc[answers]

,date,title,summary,text,url
18448,2011-04-09 12:17:00,Android расправляет плечи,Рынок смартфонов увеличится на 58% в этом году...,В 2011 году в мире будет продано 468 млн смарт...,https://www.gazeta.ru/business/2011/04/08/3579...
1,2013-01-24 18:20:09,Google закончил поиск,"Юлия Соловьева, экс-директор холдинга «Профмед...",Российское подразделение интернет-корпорации G...,https://www.gazeta.ru/business/2013/01/24/4939...
40192,2012-01-19 12:30:33,Клуб инсайдерской торговли,"В США выявлено крупнейшее мошенничество, совер...","Комиссия по ценным бумагам США, ФБР и прокурат...",https://www.gazeta.ru/business/2012/01/19/3967...


**Выводы:** 
- Просто берт из коробки работает намного лучше Word2Vec и FastText, темы действительно выделяются довольно хорошо.
- Чтобы улучшить поиск, можно использовать различные способы вычисления меры близости, и даже надстраивать отдельные сети для ее определения. Также можно дообучить берт под конкретную задачу или датасет.